# Game Artificial Intelligence Methods

Creator: Mikołaj Marmurowicz


## SUDOKU


SUDOKU can be interpreted as a constraint satisfaction problem. We can represent simple CSP problems as follows.

In [1]:
class CSP:
    def __init__(self):
        self.domains = {}
        self.binary = []
    
    def addVariable(self, var, domain):        
        assert var not in self.domains
        self.domains[var] = set(domain)
        
    def addBinaryConstraint(self, var1, operator, var2):
        assert var1 in self.domains
        assert var2 in self.domains
        c = (var1, operator, var2)
        self.binary.append(c)      
        
    def verify(self, left, operator, right):
        if operator[0] == '=':
            return left == right
        if operator == '!=':
            return left != right
        if operator == '<':
            return left < right
        if operator == '<=':
            return left <= right
        if operator == '>':
            return left > right
        if operator == '>=':
            return left >= right
        
    def is_complete(self, assignment):
        return self.domains.keys() <= assignment.keys() 
        
    def is_consistent(self, assignment):
        for var, value in assignment.items():            
            if value not in self.domains[var]:
                return False
        for var1, op, var2 in self.binary:
            if var1 in assignment and var2 in assignment:
                if not self.verify(assignment[var1], op, assignment[var2]):
                    return False
        return True

A SUDOKU game is represented as follows:

In [2]:
puzzle = '''
__3_2_6__
9__3_5__1
__18_64__
__81_29__
7_______8
__67_82__
__26_95__
8__2_3__9
__5_1_3__
'''

solution = '''
483921657
967345821
251876493
548132976
729564138
136798245
372689514
814253769
695417382
'''

Now given the previous CSP definition we may represent SUDOKU as a CSP problem.

In [3]:
sudoku = CSP()

iterator = 0
for letter in puzzle:
    i = iterator // 9
    j = iterator % 9
    if letter == "_":
        sudoku.addVariable(chr(ord('A') + i)+str(j+1), {1,2,3,4,5,6,7,8,9})
        iterator += 1
    elif letter in [str(i) for i in range(1,10)]:
        sudoku.addVariable(chr(ord('A') + i)+str(j+1), {int(letter)})
        iterator += 1

for letter in 'ABCDEFGHI':
    for number in range(1, 10):
        for number2 in range(number + 1, 10):
            sudoku.addBinaryConstraint(letter+str(number), '!=', letter+str(number2))
            
for number in range(1, 10):
    for count, letter in enumerate('ABCDEFGHI'):
        for letter2 in 'ABCDEFGHI'[count+1:]:
            sudoku.addBinaryConstraint(letter+str(number), '!=', letter2+str(number))

for iteration in range(9):
    i = iteration // 3
    j = iteration % 3
    for letter in [chr(ord('A') + 3*i), chr(ord('B') + 3*i), chr(ord('C') + 3*i)]:
        for number in [1 + 3*j, 2 + 3*j, 3 + 3*j ]:
            for letter2 in [chr(ord('A') + 3*i), chr(ord('B') + 3*i), chr(ord('C') + 3*i)]:
                for number2 in [1 + 3*j, 2 + 3*j, 3 + 3*j ]:
                    if (letter2 >= letter and number2 > number) or (letter2 >letter):
                        sudoku.addBinaryConstraint(letter+str(number), '!=', letter2+str(number2))

### Recursive Backtracking with Heuristics.

For the following problem Backtracking algorithm would take way too much time, thus I have implemented a Recursive Backtracking with the minimum remaining value and degree heuristics.

In [4]:
class RecursiveBacktrackingWithHeuristics:
    def __init__(self, csp):
        self.csp = csp
        self.counter = 0
        
    def select_unassigned_variable(self, assignment):
        variables = []
        for var in self.csp.domains.keys():
            if var not in assignment:
                MRV = 0
                for value in self.order_domain_values(var, assignment):
                    assignment[var] = value
                    if self.csp.is_consistent(assignment):
                        MRV += 1
                    assignment.pop(var)
                variables.append([-MRV, var])
        curr_max = max(variables)[0]
        more_than_one = [var[1] for var in variables if var[0] == curr_max]
        if len(more_than_one) == 1:
            return more_than_one[0]
        out = dict.fromkeys(more_than_one, 0)
        for constrain in self.csp.binary:
            if constrain[0] in more_than_one:
                out[constrain[0]] += 1
            if constrain[2] in more_than_one:
                out[constrain[2]] += 1
        return max(out, key=lambda key: out[key])

    def order_domain_values(self, variable, assignment):
        return self.csp.domains[variable]
        
    def solve(self):
        self.counter = 0
        assignment = {}
        return self.backtracking(assignment)
    
    def backtracking(self, assignment):
        if self.csp.is_complete(assignment):
            return assignment
        unassigned = self.select_unassigned_variable(assignment)
        for value in self.order_domain_values(unassigned, assignment):
            assignment[unassigned] = value
            self.counter += 1
            if self.csp.is_consistent(assignment):
                result = self.backtracking(assignment)
                if result != None:
                    return result
                assignment.pop(unassigned)
        return None

A method to print the solution.

In [5]:
def printing(solution):
    string = ""
    iterator = 1
    sorted_solution = dict(sorted(solution.items()))
    for key, value in sorted_solution.items():
        if iterator % 9 == 0:
            string += str(value) + "\n"
        else:
            string += str(value)
        iterator += 1
    print(string)

Now let us see, if the given method provides satisfactory results.

In [6]:
solver = RecursiveBacktrackingWithHeuristics(sudoku)
assignment = solver.solve()
print("Assignment")
printing(assignment)
print("Is consistent?", sudoku.is_consistent(assignment))
print("Is complete?", sudoku.is_complete(assignment))
print("Number of considered assignments", solver.counter) 

Assignment
483921657
967345821
251876493
548132976
729564138
136798245
372689514
814253769
695417382

Is consistent? True
Is complete? True
Number of considered assignments 278


We can see that the solution is correct, but the number of considered assignments can still be lowered. Therefore, now I will add the AC3 (Arc consistency 3) algorithm on top of the previous model.

### Recursive Backtracking with Heuristics and Arc Consistency 3

In [7]:
class RecursiveBacktrackingWithAC3:
    def __init__(self, csp):
        self.csp = csp
        self.counter = 0
        
    def findNeighbours(self, variable):
        neighbours = []
        for constrain in self.csp.binary:
            if (constrain[0] == variable):
                neighbours.append(constrain[2])
            elif (constrain[2] == variable):
                neighbours.append(constrain[0])
        return list(set(neighbours))
                
    def ac3(self):
        queue = list(self.csp.binary)
        while queue:
            test = queue.pop(0)
            xi = test[0]
            xj = test[2]
            if self.removeInconsistentValues(xi, xj):
                neighbours = self.findNeighbours(xi)
                for xk in neighbours:
                    if xk != xi:
                        queue.append((xk,'!=', xi))
        return

    def removeInconsistentValues(self, xi, xj):
        revised = False
        for x in self.csp.domains[xi].copy():
            if not any([x != y for y in self.csp.domains[xj]]):
                self.csp.domains[xi].remove(x)
                revised = True
        return revised
      
    def solve(self):
        self.counter = 0
        assignment = {}
        return self.backtracking(assignment)
    
    def select_unassigned_variable(self, assignment):
        variables = []
        for var in self.csp.domains.keys():
            if var not in assignment:
                MRV = 0
                for value in self.order_domain_values(var, assignment):
                    assignment[var] = value
                    if self.csp.is_consistent(assignment):
                        MRV += 1
                    assignment.pop(var)
                variables.append([-MRV, var])
        curr_max = max(variables)[0]
        more_than_one = [var[1] for var in variables if var[0] == curr_max]
        if len(more_than_one) == 1:
            return more_than_one[0]
        out = dict.fromkeys(more_than_one, 0)
        for constrain in self.csp.binary:
            if constrain[0] in more_than_one:
                out[constrain[0]] += 1
            if constrain[2] in more_than_one:
                out[constrain[2]] += 1
        return max(out, key=lambda key: out[key])
    
    def order_domain_values(self, variable, assignment):
        return self.csp.domains[variable]
    
    def backtracking(self, assignment):
        if self.csp.is_complete(assignment):
            return assignment
        unassigned = self.select_unassigned_variable(assignment)
        for value in self.order_domain_values(unassigned, assignment):
            assignment[unassigned] = value
            self.ac3()
            self.counter += 1
            if self.csp.is_consistent(assignment):
                result = self.backtracking(assignment)
                if result != None:
                    return result
                assignment.pop(unassigned)
        return None  

Now let us test this solution with the new model.

In [8]:
solver = RecursiveBacktrackingWithAC3(sudoku)
assignment = solver.solve()
print("Assignment")
printing(assignment)
print("Is consistent?", sudoku.is_consistent(assignment))
print("Is complete?", sudoku.is_complete(assignment))
print("# considered assignments", solver.counter)

Assignment
483921657
967345821
251876493
548132976
729564138
136798245
372689514
814253769
695417382

Is consistent? True
Is complete? True
# considered assignments 195


We can see that the number of considered assignments was successfully lowered.

## Tic-Tac-Toe

The other considered games can be represented in terms of the abstract class presented below.

In [9]:
class Game:
    @property
    def initial_state(self):
        ...
        return state
    
    def player(self, state):
        ...
        return playerno
        
    def actions(self, state):
        ...
        return actions
        
    def result(self, state, action):
        ...
        return new_state
        
    def is_terminal(self, state):
        ...
        return boolean
        
    def utility(self, state, player):
        ...        
        return number
        
    def print_state(self, state):
        ...        

Now let us represent the tic-tac-toe game in terms of the abstract game class.

In [17]:
def opponent(player):    
        assert player in {1, 2}
        if player == 1:
            return 2
        else:
            return 1
        

class TicTacToe(Game):    
    @property
    def initial_state(self):
        return (1, (0,)*9)
    
    def player(self, state):
        return state[0]
        
    def actions(self, state):
        return [i for i, v in enumerate(state[1]) if v == 0]
        
    def result(self, state, action):
        board = state[1]
        assert board[action] == 0
        assert state[0] in {1, 2}
        board = board[:action] + (state[0],) + board[action+1:]
        next_player = opponent(state[0])        
        return (next_player, board)
        
    def _has_line(self, state, player):
        board = state[1]
        for i in [0, 3, 6]:
            if board[i] == board[i+1] == board[i+2] == player:
                return True
        for i in [0, 1, 2]:
            if board[i] == board[i+3] == board[i+6] == player:
                return True
        if board[0] == board[3+1] == board[2*3+2] == player:
            return True
        if board[2] == board[3+1] == board[2*3] == player:
            return True
        return False
        
    def is_terminal(self, state):
        if all([v != 0 for v in state[1]]):
            return True
        return self._has_line(state, 1) or self._has_line(state, 2)
    
    def utility(self, state, player):
        assert player in {1, 2}
        mine = self._has_line(state, player)
        opponents = self._has_line(state, opponent(player))
        if mine and not opponents:
            return 1
        if not mine and opponents:
            return -1
        return 0    
    
    def print_state(self, state):
        print("Player making move", " OX"[state[0]])
        board = ["_OX"[v] for v in state[1]]
        print(*board[0:3])
        print(*board[3:6])
        print(*board[6:9])

Additionally, let us define a dummy player that will choose the first available move and a judge that will choose the winner of the encounter.

In [18]:
def dummy(game, state):
    return game.actions(state)[0]


def judge(game: Game, player1, player2):    
    state = game.initial_state

    while not game.is_terminal(state):
        if game.player(state) == 1:
            action = player1(game, state)
        else:
            action = player2(game, state)        
        game.print_state(state)
        print("Action:", action)
        print()
        state = game.result(state, action)

    game.print_state(state)
    print("Reached terminal state?", game.is_terminal(state))
    u1 = game.utility(state, 1)
    u2 = game.utility(state, 2)
    print("Utility for the 1st player", u1)
    print("Utility for the 2nd player", u2)
    if u1 > u2:
        print("Winner: 1st player")
    elif u1 < u2:
        print("Winner: 2nd player")
    else:
        print("Draw")

### Minimax algorithm.

In [19]:
def minimax(game: Game, state):
    player = game.player(state)
    value, move = maxValue(game, state, player)
    return move

def maxValue(game, state, player):
    if game.is_terminal(state):
        return game.utility(state, player), None
    v = -999999999
    for action in game.actions(state):
        v2, a2 = minValue(game, game.result(state, action), player)
        if v2 > v:
            v, move = v2, action
    return v, move

def minValue(game, state, player):
    if game.is_terminal(state):
        return game.utility(state, player), None
    v = 999999999
    for action in game.actions(state):
        v2, a2 = maxValue(game, game.result(state, action), player)
        if v2 < v:
            v, move = v2, action
    return v, move

Now let us test if the minimax algorithm performs better than the dummy.

In [20]:
%time judge(TicTacToe(), minimax, dummy)

Player making move O
_ _ _
_ _ _
_ _ _
Action: 0

Player making move X
O _ _
_ _ _
_ _ _
Action: 1

Player making move O
O X _
_ _ _
_ _ _
Action: 3

Player making move X
O X _
O _ _
_ _ _
Action: 2

Player making move O
O X X
O _ _
_ _ _
Action: 4

Player making move X
O X X
O O _
_ _ _
Action: 5

Player making move O
O X X
O O X
_ _ _
Action: 6

Player making move X
O X X
O O X
O _ _
Reached terminal state? True
Utility for the 1st player 1
Utility for the 2nd player -1
Winner: 1st player
CPU times: total: 266 ms
Wall time: 1.36 s


We can see that the first player won, thus minimax performed better than dummy player. But this method can still be improved in terms of efficiency.

### Alpha-Beta Algorithm.

In [21]:
def alphabeta(game, state):
    player = game.player(state)
    value, move = maxValue(game, state, player, -999999999, 999999999)
    return move

def maxValue(game, state, player, alpha, beta):
    if game.is_terminal(state):
        return game.utility(state, player), None
    v = -999999999
    for action in game.actions(state):
        v2, a2 = minValue(game, game.result(state, action), player, alpha, beta)
        if v2 > v:
            v, move = v2, action
            alpha = max(alpha, v)
        if v >= beta:
            return v, move
    return v, move

def minValue(game, state, player, alpha, beta):
    if game.is_terminal(state):
        return game.utility(state, player), None
    v = 999999999
    for action in game.actions(state):
        v2, a2 = maxValue(game, game.result(state, action), player, alpha, beta)
        if v2 < v:
            v, move = v2, action
            beta = min(beta, v)
        if v <= beta:
            return v, move
    return v, move

Now let us test this solution.

In [22]:
%time judge(TicTacToe(), alphabeta, dummy)

Player making move O
_ _ _
_ _ _
_ _ _
Action: 0

Player making move X
O _ _
_ _ _
_ _ _
Action: 1

Player making move O
O X _
_ _ _
_ _ _
Action: 2

Player making move X
O X O
_ _ _
_ _ _
Action: 3

Player making move O
O X O
X _ _
_ _ _
Action: 4

Player making move X
O X O
X O _
_ _ _
Action: 5

Player making move O
O X O
X O X
_ _ _
Action: 6

Player making move X
O X O
X O X
O _ _
Reached terminal state? True
Utility for the 1st player 1
Utility for the 2nd player -1
Winner: 1st player
CPU times: total: 0 ns
Wall time: 5.44 ms


We can see that the Wall time and CPU time is much lower than in the minimax algorithm.

This game is not particularly difficult, therefore we may consider a different game to really test these algorithms.

## Migration

The Migration game rules can be seen here: https://www.di.fc.ul.pt/~jpn/gv/migration.htm. This game can be represented as follows.

In [23]:
import math

class Migration:
    def __init__(self, n):
        self.n = n
    
    @property
    def initial_state(self):
        board = [[0]*self.n for _ in range(self.n)]
        k = math.ceil(self.n/2 - 1)
        for y in range(k):
            for x in range(y + 1, self.n - y - 1):
                board[x][y] = 1    
        for x in range(k):
            for y in range(x + 1, self.n - x - 1):
                board[self.n - x - 1][y] = 2
        board = tuple((tuple(row) for row in board))
        return (1, board)
    
    def player(self, state):
        return state[0]
    
    def _is_valid(self, x, y):
        return 0 <= x < self.n and 0 <= y < self.n
    
    def actions(self, state):
        board = state[1]
        player = self.player(state)
        opp = opponent(player)
        if player == 1:
            dx, dy = 0, 1
        else:
            assert player == 2
            dx, dy = -1, 0
        actions = []
        for x in range(self.n):
            nx = x + dx
            for y in range(self.n):
                ny = y + dy
                if board[x][y] == player and self._is_valid(nx, ny) and board[nx][ny] == 0:
                    actions.append((x, y, nx, ny))
        return actions
    
    def result(self, state, action):
        x, y, nx, ny = action
        player, board = state
        board = [list(row) for row in board]
        assert board[x][y] == player
        assert board[nx][ny] == 0
        board[x][y] = 0
        board[nx][ny] = player
        board = tuple((tuple(row) for row in board))
        return (opponent(player), board)
    
    def is_terminal(self, state):
        return len(self.actions(state)) == 0
        
    def utility(self, state, player):
        assert self.is_terminal(state)
        if self.player(state) == player:
            return -1
        else:
            return 1
        
    def print_state(self, state):
        print("Player making move", "_\u25CB\u25CF"[state[0]])
        for row in state[1]:
            print(*["_\u25CB\u25CF"[v] for v in row]) 

Now let us test how alpha-beta agents perform on the Migration game with 5 X 5 board.

In [24]:
%time judge(Migration(5), alphabeta, alphabeta)

Player making move ○
_ _ _ _ _
○ _ _ _ _
○ ○ _ _ _
○ _ ● _ _
_ ● ● ● _
Action: (1, 0, 1, 1)

Player making move ●
_ _ _ _ _
_ ○ _ _ _
○ ○ _ _ _
○ _ ● _ _
_ ● ● ● _
Action: (3, 2, 2, 2)

Player making move ○
_ _ _ _ _
_ ○ _ _ _
○ ○ ● _ _
○ _ _ _ _
_ ● ● ● _
Action: (1, 1, 1, 2)

Player making move ●
_ _ _ _ _
_ _ ○ _ _
○ ○ ● _ _
○ _ _ _ _
_ ● ● ● _
Action: (4, 1, 3, 1)

Player making move ○
_ _ _ _ _
_ _ ○ _ _
○ ○ ● _ _
○ ● _ _ _
_ _ ● ● _
Action: (1, 2, 1, 3)

Player making move ●
_ _ _ _ _
_ _ _ ○ _
○ ○ ● _ _
○ ● _ _ _
_ _ ● ● _
Action: (2, 2, 1, 2)

Player making move ○
_ _ _ _ _
_ _ ● ○ _
○ ○ _ _ _
○ ● _ _ _
_ _ ● ● _
Action: (1, 3, 1, 4)

Player making move ●
_ _ _ _ _
_ _ ● _ ○
○ ○ _ _ _
○ ● _ _ _
_ _ ● ● _
Action: (1, 2, 0, 2)

Player making move ○
_ _ ● _ _
_ _ _ _ ○
○ ○ _ _ _
○ ● _ _ _
_ _ ● ● _
Action: (2, 1, 2, 2)

Player making move ●
_ _ ● _ _
_ _ _ _ ○
○ _ ○ _ _
○ ● _ _ _
_ _ ● ● _
Action: (3, 1, 2, 1)

Player making move ○
_ _ ● _ _
_ _ _ _ ○
○ ● ○ _ _
○ _ _ _ _
_ _ ● ● _

We can notice that the time is much higher than in tic-tac-toe, thus some improvements can be implemented.

### Alpha-beta with limited depth and heuristics.

The heuristic below counts the number of available moves for the player after making a move and adds a point for each blocked opponent's pawn. 

In [25]:
class MigrationWithHeuristic(Migration):
    def evaluate(self, state, player):
        number = 0
        n = len(state)
        k = math.ceil(n/2 - 1)
        if player == 2:
            for i in range(n):
                for j in range(n):
                    if state[i][j] == 2:
                        if i - 1 != -1:
                            if state[i - 1][j] == 0:
                                number += 1
                    elif state[i][j] == 1:
                        if j +1 != n:
                            if state[i][j + 1] != 0:
                                number += 1
        elif player == 1:
            for i in range(n):
                for j in range(n):
                    if state[i][j] == 1:
                        if j + 1 != n:
                            if state[i][j + 1] == 0:
                                number += 1
                    elif state[i][j] == 2:
                        if i - 1 != -1:
                            if state[i - 1][j] != 0:
                                number += 1
        return number

Now let us implement the alpha-beta with limited depth and heuristic function defined above.

In [26]:
class HeuristicAlphaBeta:
    def __init__(self, max_depth):
        self.max_depth = max_depth
        self.counter = 0
        
    def __call__(self, game, state):
        best_move = self.alphabeta(game, state)
        return best_move
    
    def alphabeta(self, game, state):
        player, curr_state = state
        self.counter = 0
        value, move = self.maxValue(game, state, player, -999999999, 999999999)
        return move

    def maxValue(self, game, state, player, alpha, beta):
        self.counter += 1
        if game.is_terminal(state):
            return game.utility(state, player), None
        v = -999999999
        for action in game.actions(state):
            if self.counter + 1 >= self.max_depth:
                return MigrationWithHeuristic.evaluate(self, game.result(state, action)[1], player), action
            else :
                v2, a2 = self.minValue(game, game.result(state, action), player, alpha, beta)
            if v2 > v:
                v, move = v2, action
                alpha = max(alpha, v)
            if v >= beta:
                self.counter -= 1
                return v, move
        self.counter -= 1
        return v, move

    def minValue(self, game, state, player, alpha, beta):
        self.counter += 1
        if game.is_terminal(state):
            return game.utility(state, player), None
        v = 999999999
        for action in game.actions(state):
            if self.counter + 1 >= self.max_depth:
                return MigrationWithHeuristic.evaluate(self, game.result(state, action)[1], player), action
            else:
                v2, a2 = self.maxValue(game, game.result(state, action), player, alpha, beta)
            if v2 < v:
                v, move = v2, action
                beta = min(beta, v)
            if v <= beta:
                self.counter -= 1
                return v, move
        self.counter -= 1
        return v, move

Now let us test the time of evaluation for the heuristic alpha-betas with depth = 2 and with a board of size 5 X 5.

In [27]:
%time judge(MigrationWithHeuristic(5), HeuristicAlphaBeta(2), HeuristicAlphaBeta(2))

Player making move ○
_ _ _ _ _
○ _ _ _ _
○ ○ _ _ _
○ _ ● _ _
_ ● ● ● _
Action: (1, 0, 1, 1)

Player making move ●
_ _ _ _ _
_ ○ _ _ _
○ ○ _ _ _
○ _ ● _ _
_ ● ● ● _
Action: (3, 2, 2, 2)

Player making move ○
_ _ _ _ _
_ ○ _ _ _
○ ○ ● _ _
○ _ _ _ _
_ ● ● ● _
Action: (1, 1, 1, 2)

Player making move ●
_ _ _ _ _
_ _ ○ _ _
○ ○ ● _ _
○ _ _ _ _
_ ● ● ● _
Action: (4, 1, 3, 1)

Player making move ○
_ _ _ _ _
_ _ ○ _ _
○ ○ ● _ _
○ ● _ _ _
_ _ ● ● _
Action: (1, 2, 1, 3)

Player making move ●
_ _ _ _ _
_ _ _ ○ _
○ ○ ● _ _
○ ● _ _ _
_ _ ● ● _
Action: (2, 2, 1, 2)

Player making move ○
_ _ _ _ _
_ _ ● ○ _
○ ○ _ _ _
○ ● _ _ _
_ _ ● ● _
Action: (1, 3, 1, 4)

Player making move ●
_ _ _ _ _
_ _ ● _ ○
○ ○ _ _ _
○ ● _ _ _
_ _ ● ● _
Action: (1, 2, 0, 2)

Player making move ○
_ _ ● _ _
_ _ _ _ ○
○ ○ _ _ _
○ ● _ _ _
_ _ ● ● _
Action: (2, 1, 2, 2)

Player making move ●
_ _ ● _ _
_ _ _ _ ○
○ _ ○ _ _
○ ● _ _ _
_ _ ● ● _
Action: (3, 1, 2, 1)

Player making move ○
_ _ ● _ _
_ _ _ _ ○
○ ● ○ _ _
○ _ _ _ _
_ _ ● ● _

We can see that the time of evaluation has been drastically lowered, but the quality has also decreased slightly. Thus a trade-off between considered depth and the speed of the algorithm needs to be considered.